In [1]:
#we got an acess token and an expiration, these items are state items, they'll change over time. so we need to create a class that'll respond to these changes, so that we can make our API calls through that class, instead of using functions

In [2]:
!pip install requests

In [3]:
import requests
import datetime

In [4]:
import base64

In [5]:
import configparser
config = configparser.ConfigParser()
config.read('/Users/ruby/Desktop/VS-Code-Basics/credentials.ini') #PATH TO MY config file on my local device

cred = config["SPOTIFY"]
client_id = cred["CLIENT_ID"]
client_secret = cred["CLIENT_ID_SECRET"]

#print(client_id)
#print (client_secret)

In [17]:
class SpotifyAPI(object):
    access_token = None
    acess_token_expires = datetime.datetime.now()
    acess_token_did_expire = True
    client_id = None
    client_secret = None
    token_url = 'https://accounts.spotify.com/api/token'
    
    def __init__(self, client_id, client_secret, *args, **kwargs):
        super().__init__(*args, **kwargs) #it calls any class it is inherting from itself, this is used for changibility of code or you want to inherit from another class
        self.client_id = client_id
        self.client_secret = client_secret
        
    def get_client_credentials(self):
        """
        Returns a base64 encoded string
        """
        client_id = self.client_id
        client_secret = self.client_secret
        if client_secret == None or client_id == None:
            raise Exception("You must set client_id and client_secret")
        client_creds = f"{client_id}:{client_secret}"
        client_creds_b64 = base64.b64encode(client_creds.encode())
        return client_creds_b64.decode()
    
    def get_token_headers (self):
        client_creds_b64 = self.get_client_credentials()
        return {
            "Authorization": f"Basic {client_creds_b64}"
        }
    
    def get_token_data (self):
        return {
            "grant_type": "client_credentials"
        }
    
    def perform_auth (self):
        token_url = self.token_url
        token_data = self.get_token_data()
        token_headers = self.get_token_headers()
        r = requests.post (token_url, data = token_data, headers= token_headers)
        if r.status_code not in range (200,299):
            return False
        data = r.json()
        now = datetime.datetime.now()
        access_token = data['access_token']
        expires_in = data['expires_in']#time given in seconds
        expires = now + datetime.timedelta(seconds = expires_in)
        self.access_token = access_token
        self.acess_token_expires = expires
        self.access_token_did_expire = expires < now #gives a true or false value to determine when you need to refresh your token
        return True
    

In [21]:
spotify = SpotifyAPI (client_id, client_secret)

In [22]:
spotify.perform_auth()

True

In [23]:
spotify.access_token

'BQA5rBUBmABL28a2ktjo5ijkgS1F86bU_8CP7dxr8eld6coyab53aMZVCEROmLihqf8gTzv7-qJTVc1HvTA'

In [ ]:
#spotify.search
#could improve on perform_auth. like a function handler